# Local obstacle avoidance

While navigating, Thymio uses its proximity sensors to avoid physical obstacles that can be put in its path at any point and any moment. Our obstacle avoidance algorithm works for rectangular parallelepipeds and cylinders.
When Thymio detects a local obstacle, our program immediatly enters in local navigation mode.
First, we have a function "obstacle_avoidance" that manages the local avoidance problem. We decided to make the robot avoid an obstacle by always following its left side. Hence, if the robot detects the obstacle with its left sensors, it must rotate in order to follow correctly the left side of the wall. At this point, it rotates again until it does not detect the obstacle anymore. Now, it is ready to advance and rotate until it crosses the global path. Actually, we make use of another constructor: "check_global_obstacles_and_global_path" that checks if the robot reaches a map limit, a global obstacle, or the optimal path. Thus, after getting the last position of the robot, the algorithm checks if we have reached a map limit or if a global obstacle is nearby (through the occupancy grid). If this is the case, it returns obstacle=True and "obstacle_avoidance" adjusts the robot's position. Then we get the previous and the following vertices of the optimal path with respect to the actual position of the robot. In this way we can compute the straight line between these coordinates and thus verifying if Thymio reached the global path. If yes, the function returns global_path=True and the algorithm switches to the global navigation mode.

In [ ]:
#Install the tdmclient package:
!pip install tdmclient

In [1]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
from timer import *
from motion import *

# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

import numpy as np
from threading import Timer
import math

In [2]:
%%run_python

state = 1

@onevent
def move(speed_left, speed_right):
    global motor_left_target, motor_right_target, state
    if state == 1 :
        motor_left_target = speed_left
        motor_right_target = speed_right
    
@onevent 
def button_center():
    global state, motor_left_target, motor_right_target
    state = 0
    motor_left_target = 0
    motor_right_target = 0

In [3]:
# New send_command to verify that the previous command is finished

def send_command():
    global step, rt, command, ok
    if step < len(command) :
        if command[step][0] == command[step][1] and command[step][0] != 0:
            send_event("move", command[step][0] + 2, command[step][1] - 2)
        else :
            send_event("move", command[step][0], command[step][1])
            
        step += 1
    else :
        ok = 1
    return ok

In [4]:
#Constants
Ts = 0.1
thymio_speed_to_mms = 0.36
thymio_speed_to_rads = 0.0071


def move_forward(dist):
    time = abs(dist) / (thymio_speed_to_mms * 100)
    size_command = int(time / Ts)
    commands = []
    
    for i in range(size_command) :
        commands.append([100, 100])
    
    return commands
    
def move_backward(dist):
    time = abs(dist) / (thymio_speed_to_mms * 100)
    size_command = int(time / Ts)
    commands = []
    
    for i in range(size_command) :
        commands.append([-100, -100])
    
    return commands
    
def turn_left(angle):
    time = abs(angle) / (thymio_speed_to_rads * 100)
    size_command = int(time / Ts)
    commands = []
    
    for i in range(size_command) :
        commands.append([-100, 100])
    
    return commands
    
def turn_right(angle):
    time = abs(angle) / (thymio_speed_to_rads * 100)
    size_command = int(time / Ts)
    commands = []
    
    for i in range(size_command) :
        commands.append([100, -100])
    
    return commands

In [5]:
#LOCAL OBSTACLE AVOIDANCE

# get the sensor measurements
@tdmclient.notebook.sync_var
def get_proximity():
    return prox_horizontal


def check_global_obstacles_and_global_path(length_advance, final_occupancy_grid, full_path, x_est):
    """
    Check if
     - there is an obstacle or
     - there is a map limit or
     - if it crossed the end of the path
    :param length_advance: distance of advancement to check
    :param final_occupancy_grid: map grid with obstacle size increase (1: obstacle, 0: no obstacle)
    :param full_path: list of the two coordinates of the optimal path before and after actual thymio's position
    :return: A boolean to know if there is an obstacle/map limit and a boolean to know if the A* path was crossed
    """

    # Initialization of the variables
    global_path = False
    obstacle = False
    DIST_ERROR = 30

    # Get the last position of the robot
    x = x_est[0][0]
    y = x_est[1][0]
    theta = x_est[2][0]
    conv = 0.0174 # conversion deg to rad

    # Find the direction of advance to check the occupancy grid and global obstacles
    if (theta >= -22.5*conv) and (theta < 22.5*conv):  # 1
        dir_x = 1
        dir_y = 0
    elif (theta >= 22.5*conv) and (theta < 67.5*conv):  # 2
        dir_x = 1
        dir_y = 1
    elif (theta >= 67.5*conv) and (theta < 112.5*conv):  # 3
        dir_x = 0
        dir_y = 1
    elif (theta >= 112.5*conv) and (theta < 157.5*conv):  # 4
        dir_x = -1
        dir_y = 1
    elif ((theta >= 157.5*conv) and (theta <= 181*conv)) or ((theta >= -181*conv) and (theta < -157.5*conv)):  # 5
        dir_x = -1
        dir_y = 0
    elif (theta >= -157.5*conv) and (theta < -112.5*conv):  # 6
        dir_x = -1
        dir_y = -1
    elif (theta >= -112.5*conv) and (theta < -67.5*conv):  # 7
        dir_x = 0
        dir_y = -1
    elif (theta >= -67.5*conv) and (theta < -22.5*conv):  # 8
        dir_x = 1
        dir_y = -1
    else:
        dir_x = 0
        dir_y = 0
        print(" angle is not between -180 and 180 degrees")

    # compute the next step
    x_next_step = x + length_advance * dir_x
    y_next_step = y + length_advance * dir_y

    # Verify the map limits and return obstacle=True if it reaches the limit
    if (x_next_step > 719) or (x_next_step < 1) or (y_next_step > 1279) or (y_next_step < 1):
        obstacle = True
        return obstacle, global_path

    # Verify the map obstacles in 2D and return obstacle=True if it reached it
    elif final_occupancy_grid[x_next_step][y_next_step] == 1:
        obstacle = True
        return obstacle, global_path

    # Compute if we reach the global path (with an error margin)

    # First, let's get the previous and the next coordinates of the full path wrt the actual position of the robot
    x_prev = full_path[0][0]
    y_prev = full_path[0][1]
    x_next = full_path[1][0]
    y_next = full_path[1][1]

    # Let's now compute the straight line between the previous vertices and the following ones
    if (x_next-x_prev) != 0:
        slope = (y_next - y_prev) / (x_next - x_prev) # slope
        intercept = y_prev - slope * x_prev # intercept
        
        evaluation = np.abs(slope * x + intercept - y) # check if we reached the global path
        if evaluation < DIST_ERROR:  # margin of error in mm
            global_path = True
            return obstacle, global_path
            
    elif (np.abs(x-x_prev) < DIST_ERROR): # if the x is almost equal to the other two x's coordinates: we are again on the global path
        global_path = True
        return obstacle, global_path

    return obstacle, global_path


def obstacle_avoidance(full_path, final_occupancy_grid):
    """
    Obstacle avoidance: it handles the obstacle avoidance routine
    :return: a boolean to know if the A* path is crossed
    """

    """
   Constructor for initialization
   :param full_path: list of the two coordinates of the optimal path before and after actual thymio's position
   :param final_occupancy_grid: map grid with obstacle size increase (1: obstacle, 0: no obstacle)
   """
    
    #Initialization
    global_path = False
    
    # global variables
    global command, step, ok

    # Define constants
    MIN_WALL_THRESHOLD = 2000  # constant min threshold to detect a wall or not
    MAX_WALL_THRESHOLD = 3000  # constant max threshold to detect a wall or not
    conv = 0.0174 # conversion deg to rad
    DISTANCE_ADVANCE = 30 # constant step to move forward in mm
    DISTANCE_CHECK = 2 # constant step to check the occupancy grid 
    # angles of rotation
    ANGLE_AVOIDANCE = 20*conv
    ANGLE_ADJUSTMENT = 30*conv

    # Since we need to follow the wall on its left, if the robot detects an obstacle with its left sensors
    # it must rotate to avoid correctly the wall

    # If the robot detects the obstacle with its right sensor ok, it can start avoidance

    # If it detects the obstacle on the left side of the robot -> need to rotate
    sensor_values = get_proximity()  # load the five sensors values
    while (sensor_values[1] > MIN_WALL_THRESHOLD):
        step = 0
        ok = 0
        command = turn_left(ANGLE_AVOIDANCE) + [[0, 0]] # rotate left
        rt = RepeatedTimer(Ts, send_command)
        ok = send_command() # when ok = 1 the command is finished -> can execute the other command
        sensor_values = get_proximity()
    
    ok = 1
        
    # If the robot goes straight toward the corner of an obstacle, it has to adjust its angle to avoid the obstacle
    if ok: # the previous command is finished: can execute the following one
        sensor_values = get_proximity()
        while (sensor_values[2] > MIN_WALL_THRESHOLD):
            step = 0
            ok = 0
            command = turn_left(ANGLE_AVOIDANCE) + [[0, 0]]  # adjusts its angle
            rt = RepeatedTimer(Ts, send_command)
            ok = send_command()
            sensor_values = get_proximity()
        
    # The robot rotates on itself until it doesn't detect the obstacle with the extreme right sensor 
    if ok:
        sensor_values = get_proximity()
        while (sensor_values[4] > MIN_WALL_THRESHOLD):
            step = 0
            ok = 0
            command = turn_left(ANGLE_AVOIDANCE) + [[0, 0]] # rotate left
            rt = RepeatedTimer(Ts, send_command)
            ok = send_command()
            sensor_values = get_proximity()
        
    # Now the robot has to advance and rotate on the right until it finds global path: 
    # it is following the left wall of the obstacle
    sensor_values = get_proximity()
    if ok:
        while (sensor_values[4] < MAX_WALL_THRESHOLD):
            # first check global obstacles and global path
            obstacle, global_path = check_global_obstacles_and_global_path(DISTANCE_CHECK, final_occupancy_grid, full_path)
            if obstacle:  # if it finds an obstacle -> correct direction
                step = 0
                ok = 0
                command = turn_right(ANGLE_AVOIDANCE) + [[0, 0]]
                rt = RepeatedTimer(Ts, send_command)
                ok = send_command()
            if global_path:  # if the full path is reached, return -> leave the local avoidance
                return global_path
            if ok : # move forward and rotate right
                step = 0
                ok = 0
                command = move_forward(DISTANCE_ADVANCE) + turn_right(ANGLE_ADJUSTMENT)  + [[0, 0]]
                rt = RepeatedTimer(Ts, send_command)
                ok = send_command()
                sensor_values = get_proximity()
                if ok : # correct the direction: if we are rotating right too much -> rotate left
                    while (sensor_values[4] > MIN_WALL_THRESHOLD):
                        step = 0
                        ok = 0
                        command = turn_left(ANGLE_ADJUSTMENT) + [[0, 0]]
                        rt = RepeatedTimer(Ts, send_command)
                        ok = send_command()
                        sensor_values = get_proximity()
                    sensor_values = get_proximity()

    return global_path